In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("transformations_actions")

In [3]:
sc = SparkContext(conf = conf)

In [4]:
sc.getConf().getAll()
#

[('spark.master', 'local'),
 ('spark.driver.host', 'host.docker.internal'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.driver.port', '14450'),
 ('spark.app.id', 'local-1676535144402'),


In [5]:
sc.stop()

In [6]:
sc = SparkContext(conf=conf)

In [7]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면"])
# parallelize를 이용하여 RDD를 만들수있음

In [8]:
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [9]:
foods.collect()
# collect는 RDD안의 밸류를 모두 가져올수있음
# 다만, 개발 환경이나 디버깅시 사용하고, 프로덕션 환경에선 안쓰는게 좋음
# 데이터를 모두 가져와 낭비가 되며, 스파크를 쓰는 이유가 없어지기 때문

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [10]:
foods.countByValue()
# 밸류들의 값을 하나하나씩 세고 싶을때 사용


defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [11]:
foods.take(3)
# take안에 들어가는 개수의 데이터만 뽑고싶을때 사용
# 처음 순서대로 뽑힘

['짜장면', '마라탕', '짬뽕']

In [12]:
foods.take(5)

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수']

In [13]:
foods.first()
# 첫번째 데이터를 뽑고 싶을때 사용

'짜장면'

In [14]:
foods.count()
# 데이터가 총 몇개인지 셀때 사용

12

In [15]:
foods.distinct().count()
# 중복된 데이터를 빼주고 싶을땐 distinct를 사용한다
# distinct는 RDD를 리턴해주기 때문에 뒤에 개수를 셀 경우엔 count() 같은 부속 operation을 써줘야함

7

In [16]:
# foreach

In [17]:
foods.foreach(lambda x: print(x))
# 요소를 하나하나 꺼내서 함수를 적용할때 씀
# foreach는 워커노드에서 실행이 되기에 보이지 않음
# 다만 print 함수는 실행된것임

In [18]:
sc.parallelize([1, 2, 3]).map(lambda x: x + 2).collect()
# 1, 2, 3이란 RDD를 만들고 거기에 각각 2씩 더하는 함수를 적용한뒤 보여주는 과정
# collect 액션을 하지 않으면 RDD가 만들어 짐

[3, 4, 5]

In [19]:
sc.parallelize([1, 2, 3]).map(lambda x: x * 2).collect()

[2, 4, 6]

In [20]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [21]:
moviesRDD = sc.parallelize(movies)
#리스트를 RDD로 만들때 parallelize쓰면 됨

In [22]:
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [23]:
flatMovies = moviesRDD.flatMap(lambda x: x.split(" "))
# split 함수를 적용한 데이터들의 결과의 각 요소를 꺼내줌

In [24]:
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [25]:
filteredMovies = flatMovies.filter(lambda x: x != "매트릭스")
# '매트릭스'를 제외한 데이터를 뽑느 필터링 걸어주기 

In [26]:
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [27]:
num1 = sc.parallelize([1, 2, 3, 4])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [28]:
num1.intersection(num2).collect()
# intersection은 겹치는 값을 뽑아 냄

[4]

In [29]:
num1.union(num2).collect()
# union은 중복되는 값 관계없이 합쳐짐
# 일렬로 합쳐지는 걸 볼 수 있음

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [30]:
num1.subtract(num2).collect()
# substract는 intersection 값을 빼는걸 볼 수 있음
# 또한, 배열이 기본 배열과 달리 배치되는 걸 볼 수 있음

[2, 1, 3]

In [31]:
numUnion = num1.union(num2)

In [42]:
numUnion.collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [43]:
# numUnion.sample(T)
# [1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]
numUnion.sample(True, .5).collect()
# sample은 무작위로 몇개의 값을 빼올때 씀
# True는 뽑은뒤 값을 빼고 다시 넣고 샘플링 한다는 의미
# .5는 50%를 샘플링 한다는 걸 의미함

[1, 3, 4, 4, 5, 6]

In [44]:
numUnion.sample(True, .5, seed=5).collect()
# seed를 이용해서 나오는 값을 고정시킬수 있음

[1, 4, 6, 9, 9, 10]

In [35]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면", "치킨", "돈까스", "회", "햄버거", "피자"])


In [36]:
# groupBy

In [37]:
foodsGroup = foods.groupBy(lambda x: x[0])
# groupBy 무엇을 기준으로 그룹핑 할 때 씀
# 수학적으로 나눌때도 씀

In [38]:
res = foodsGroup.collect()

In [39]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
치 ['치킨']
돈 ['돈까스']
회 ['회']
햄 ['햄버거']
피 ['피자']


In [40]:
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [41]:
list(nums.groupBy(lambda x: x % 2).collect()[1][1])

[2, 4, 6, 8, 10]